In [1]:
import sys
import os
os.environ['KERAS_BACKEND'] = 'torch'
sys.path.insert(0, os.path.abspath('../'))

from TumorDetection.Utils.WorkingDir import WorkingDir
WorkingDir.set_wd()
from time import perf_counter
import torch
import numpy as np
import math
import plotly.graph_objects as go 

from TumorDetection.Utils.DictClasses import DataPathDir
from TumorDetection.Data.Loader import DataPathLoader, ImageLoader
from TumorDetection.Data.PatchDataset import PatchDataset
from TumorDetection.Models.EfficientNet.EfficientNet import EFSNetModel

In [2]:
dp = DataPathLoader(DataPathDir.get('dir_path'),
                    substring_filter='normal')
il = ImageLoader()(paths_classes=dp())

In [3]:
X_tr, X_val, y_tr, y_val = PatchDataset()(images=[t[2] for t in il],
                                          masks=[t[-1] for t in il])

In [4]:
print('Train data')
print(X_tr.shape, y_tr.shape)
perc_classes_ytr = {k: v/y_tr.size for k, v in np.asarray(np.unique(y_tr, return_counts=True)).T}
# perc_classes_ytr = [(idx, np.sum(y_tr[...,idx])/y_tr[...,idx].size) for idx in range(y_tr.shape[-1])]
print('Y Train distribution', perc_classes_ytr)
print('Weights tr:', {k: (1/math.log(1.02+v))/(1/math.log(1.02+perc_classes_ytr[0])) for k,v in perc_classes_ytr.items()})
print('Validation data')
print(X_val.shape, y_val.shape)
perc_classes_yval = {k: v/y_val.size for k, v in np.asarray(np.unique(y_val, return_counts=True)).T}
# perc_classes_yval = [(idx, np.sum(y_val[...,idx])/y_val[...,idx].size) for idx in range(y_val.shape[-1])]
print('Y Validation distribution', perc_classes_yval)
print('Weights val:', {k: (1/math.log(1.02+v))/(1/math.log(1.02+perc_classes_yval[0])) for k,v in perc_classes_yval.items()})

Train data
(10595, 256, 256, 1) (10595, 256, 256)
Y Train distribution {0: 0.7971902326122102, 1: 0.09748132623777798, 2: 0.1053284411500118}
Weights tr: {0: 1.0, 1: 5.377257825128277, 2: 5.058579544444323}
Validation data
(1625, 256, 256, 1) (1625, 256, 256)
Y Validation distribution {0: 0.836554208608774, 1: 0.06942348069411057, 2: 0.09402231069711539}
Weights val: {0: 1.0, 1: 7.22395822080262, 2: 5.730120543978491}


In [5]:
Efsmodel = EFSNetModel(verbose=1)
device = next(iter(Efsmodel.model.parameters())).device

In [6]:
device

device(type='cuda', index=0)

In [ ]:
history = Efsmodel.train_model(-X_tr, y_tr, X_val, y_val, resume_training=False)
# history = Efsmodel.train_model(torch.from_numpy(X_tr).to(device=device),
#                                torch.from_numpy(y_tr).to(device=device), 
#                                torch.from_numpy(X_val).to(device=device),
#                                torch.from_numpy(y_val).to(device=device),
#                                resume_training=False)

Epoch 1/1000
332/332 ━━━━━━━━━━━━━━━━━━━━ 0s 456ms/step - loss: 0.7260 - sparse_categorical_accuracy: 0.4572
Epoch 1: val_loss improved from inf to 2.03302, saving model to C:\Users\User\Git\TumorDetection\reporting\models\EFSNet_torch.weights.h5
332/332 ━━━━━━━━━━━━━━━━━━━━ 161s 477ms/step - loss: 0.7259 - sparse_categorical_accuracy: 0.4575 - val_loss: 2.0330 - val_sparse_categorical_accuracy: 0.6333
Epoch 2/1000
332/332 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - loss: 0.6206 - sparse_categorical_accuracy: 0.6550
Epoch 2: val_loss did not improve from 2.03302
332/332 ━━━━━━━━━━━━━━━━━━━━ 160s 476ms/step - loss: 0.6206 - sparse_categorical_accuracy: 0.6549 - val_loss: 26.8078 - val_sparse_categorical_accuracy: 0.6524
Epoch 3/1000
332/332 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - loss: 0.6142 - sparse_categorical_accuracy: 0.6349
Epoch 3: val_loss did not improve from 2.03302
332/332 ━━━━━━━━━━━━━━━━━━━━ 159s 476ms/step - loss: 0.6142 - sparse_categorical_accuracy: 0.6349 - val_loss: 2.2187 - val

In [ ]:
Efsmodel.model.predict(X_tr[0:1])

In [ ]:
batch_xtr = torch.from_numpy(X_tr[0:1])
print(type(batch_xtr))
batch_ypred = Efsmodel.model.predict(batch_xtr)
print(type(batch_ypred))